Installing the required packages for the project

Transformers -> To generate question and answers from the pretrained model

Pypdf-> To extract the text content from PDF
PDF Extraction
Extracting the content of the PDF files to text using pypdf library

PYPDF

It is one of the built-in python library to work with PDF files such as Extracting text, Merging, Editing etc...

Version: 3.16.4

Only extracting the Text hence importing PDFReader. File handling method is used to open the pdf, rb- as the file have images etc. To check the total num of pages For loop is used to read the content of all the pages in the pdf

Steps in Generating Q&A

1. Analyzing the content
2. Identifying Keywords
3. Transformers
4. Q&A generation

T5 model- Text To Text Transfer Transformer "t5-small"


In [ ]:
!pip install transformers
!pip install pypdf



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.2 MB/s eta 0:00:00


cloning the github repo to access the pdf file

In [ ]:
! git clone https://github.com/akaiketech/internship-assignment-nlp

Cloning into 'internship-assignment-nlp'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 5), reused 5 (delta 2), pack-reused 0
Receiving objects: 100% (18/18), 8.94 MiB | 13.67 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
#Importing the req pre trained model from transformer along with the Pdf extracter

from transformers import T5ForConditionalGeneration, T5Tokenizer
from pypdf import PdfReader

def text_extract(pdf_file):
    # this will check whether the file provided is in string format or not,If not raises an error.

    if not isinstance(pdf_file, str):
        raise ValueError("Invalid argument type. 'pdf_file' should be a string.")

    with open(pdf_file, 'rb') as pdf:
        read_pdf = PdfReader(pdf)
        page_nums = len(read_pdf.pages)
        text = ''
        for pages in read_pdf.pages:
            text += pages.extract_text()
    return text

# Load T5 model and tokenizer
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_mcma_question(pdf_file):
    text_content = text_extract(pdf_file)

    # Setting up the input string with placeholders(here we are using 4 placeholders)
    # content- the extratcted text will be passed in to this as a passage to generate Question,Answer,and Options

    input_string = "Question: [Question] ; Answer: [Answer] ; Options: [Options] ;  Content: [Content]"
    input_text = input_string.replace("[Content]", text_content)
    #print(input_text)


    # Tokenize and encode the input text, Encoded input will be stored in input_ids
    # have set the maximum length, the input will be taken till the set max_length
    # inputs will be truncated once the set max length is exceeded
    # By setting return_tensors="pt", the tokenizer will return the encoded inputs as PyTorch tensors

    inputs = tokenizer.encode_plus(input_text, return_tensors="pt", max_length=5000, truncation=True)
    input_ids = inputs["input_ids"]


    # taking the inputs,output will be generated.
    # Num_return_sequences and num_beams - defines the number of outputs required to generate accoridngly (here we will get 20 questions in output) and beam search width
    # tensors will be converted to string again ,conversion will happen in batch and skipping the special characters

    output = model.generate(input_ids, max_length=100, num_beams=10, num_return_sequences=10)
    decoded_outputs = tokenizer.batch_decode(output, skip_special_tokens=True)
    return decoded_outputs


pdf_file = "/content/internship-assignment-nlp/Dataset/chapter-2.pdf"
generated_output = generate_mcma_question(pdf_file)
#print(generated_output)
#print(text_content)


# Iterating the output to split the questions,answers and MCoptions which will be stored in a empty list as tuples by replacing the placeholder text.
# Printing the Questions along with its answer and MC options


mcma_ques = []

for out in generated_output:
    components = out.split(" ")
    if len(components) >= 4:
        question = components[0].strip().replace("Question:","")
        crct_ans = [a.strip() for a in components[1].replace("Answer:","").split(',')]
        options = [o.strip() for o in components[2].replace("Options:","").split(',')]

        mcma_ques.append((question,crct_ans,options))
    else:
        print(f"Invalid format encountered. Components: {components}")

for i, (question,crct_ans,options) in enumerate(mcma_ques, 1):
    print(f" {i}.  Question: {question}")
    for j, option in enumerate(options, 1):
        print(f" Options: {option}")
    print(f" Answer: {', '.join(crct_ans)}")
    print()




Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


 1.  Question: the
 Options: had
 Answer: Company

 2.  Question: the
 Options: had
 Answer: Company

 3.  Question: the
 Options: had
 Answer: Company

 4.  Question: the
 Options: had
 Answer: Company

 5.  Question: the
 Options: had
 Answer: Company

 6.  Question: the
 Options: had
 Answer: Company

 7.  Question: the
 Options: had
 Answer: Company

 8.  Question: the
 Options: had
 Answer: Company

 9.  Question: the
 Options: had
 Answer: Company

 10.  Question: the
 Options: had
 Answer: Company

